In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rp2 import hagai_2018, create_folder
from rp2.environment import check_environment
from rp2.paths import get_output_path

check_environment()

In [ ]:
def load_gene_list(csv_path, symbol_column="symbol"):
    df = pd.read_csv(csv_path)
    return df[symbol_column].values

## Export expression counts from Hagai *et al.* (2018) dataset

To export RNA counts for specific genes and conditions:
1. Ensure all previous cells have been executed
2. Edit the values of export_conditions (and show_histograms) accordingly in the cell below
3. Run the cell below

Output files will be created in *Output/Exported_Counts*.

Note that it is possible to combine conditions (i.e. replicate, treatment and time_point) by creating a list of lists. For example
```python
replicate = [["1", "2"], "3"]
```
will create results with replicates 1 and 2 combined and 3 as a separate condition.

For bulk exporting, a list of genes can be loaded from a .csv file as follows:

```python
genes = load_gene_list("Output/burst_trends_gene_list.csv")
```

In [ ]:
export_conditions = dict(
    species=["mouse"],
    count_type=["median"],  # Choose from umi, cpt, cpm or median
    genes=["tnf"],
    replicate=["1", "2", "3"],
    treatment=[["unst", "lps"], ["unst", "pic"]],
    time_point=["0", "2", "4", "6"],
)

show_histograms = True


output_path = get_output_path("Exported_Counts")
create_folder(output_path)

count_adata_map = {}

for values in itertools.product(*export_conditions.values()):
    export_dict = dict(zip(export_conditions.keys(), values))

    species = export_dict.pop("species")
    count_type = export_dict.pop("count_type")
    gene_symbols = export_dict.pop("genes").lower()

    count_key = f"{species}-{count_type}"
    if count_key in count_adata_map:
        count_adata = count_adata_map[count_key]
    else:
        print(f'Loading {species} "{count_type}" counts')
        count_adata = hagai_2018.load_counts(species, scaling=count_type)
        count_adata_map[count_key] = count_adata

    count_adata = count_adata[:, count_adata.var.symbol.str.lower() == gene_symbols]
    if count_adata.n_vars != 1:
        if count_adata.n_vars == 0:
            print(f"  No genes with symbol '{gene_symbols}'")
        else:
            print(f"  Multiple genes with symbol '{gene_symbols}':")
            for name in count_adata:
                print(f"  {name}")
        continue

    filename = f"{species}_{count_type}_{gene_symbols}"

    for k, v in export_dict.items():
        if not isinstance(v, list):
            v = [v]
        count_adata = count_adata[count_adata.obs[k].isin(v), :]

        filename += f"_{k}_" + "_".join(v)

    filename += ".txt"

    print(f"File: {filename}")
    print(f"  {len(count_adata):,} samples")

    counts = np.asarray(count_adata.X.A).squeeze().round().astype(np.int)
    np.savetxt(output_path.joinpath(filename), counts, fmt="%d")

    if show_histograms and (len(counts) > 0):
        plt.hist(counts, log=True)
        plt.xlabel("Count")
        plt.ylabel("Frequency")
        plt.show()